In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

In [9]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MLlibTrain") \
    .getOrCreate()

In [10]:
# Read csv file
data = spark.read.csv("/home/jovyan/work/ai4i2020.csv", header=True, inferSchema=True, sep=",")

data.show()

+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+---------------+---+---+---+---+---+
|UDI|Product ID|Type|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Machine failure|TWF|HDF|PWF|OSF|RNF|
+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+---------------+---+---+---+---+---+
|  1|    M14860|   M|              298.1|                  308.6|                  1551|       42.8|              0|              0|  0|  0|  0|  0|  0|
|  2|    L47181|   L|              298.2|                  308.7|                  1408|       46.3|              3|              0|  0|  0|  0|  0|  0|
|  3|    L47182|   L|              298.1|                  308.5|                  1498|       49.4|              5|              0|  0|  0|  0|  0|  0|
|  4|    L47183|   L|              298.2|                  308.6|                 

In [11]:
# Colunas de features numéricas
feature_cols = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]

# Coluna categórica a ser "dummyficada"
cat_col = "Type"

# Índice para a coluna categórica
indexer = StringIndexer(inputCol=cat_col, outputCol=cat_col+"_index")
data = indexer.fit(data).transform(data)

# One-Hot Encoding para a coluna categórica
encoder = OneHotEncoder(inputCol=cat_col+"_index", outputCol=cat_col+"_vec")
data = encoder.fit(data).transform(data)

# Agora temos um vetor codificado para "type"
# Atualizamos o conjunto de features para incluir também o type_vec
final_feature_cols = feature_cols + [cat_col+"_vec"]

assembler = VectorAssembler(inputCols=final_feature_cols, outputCol="features")
data = assembler.transform(data)

In [14]:

# Lista de falhas para prever
labels = ["TWF", "HDF", "PWF", "OSF", "RNF"]

# Divide dados em treino e teste uma única vez, se quiser usar o mesmo split
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Dicionário para guardar modelos e avaliações
models = {}
evaluations = {}

# Avaliador binário - podemos usar AUC e/ou Accuracy
# Por padrão BinaryClassificationEvaluator avalia AUC do ROC.
auc_evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")


AssertionError: 

In [13]:
# Para accuracy, precisamos de MulticlassClassificationEvaluator (mesmo sendo binário)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

for label in labels:
    # Cria um DF temporário com a coluna alvo renomeada para 'label'
    train_df = train_data.withColumnRenamed(label, "label")
    test_df = test_data.withColumnRenamed(label, "label")

    # Treina um modelo de regressão logística (binária)
    lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)
    model = lr.fit(train_df)

    # Previsões no conjunto de teste
    predictions = model.transform(test_df)

    # Avaliação
    auc = auc_evaluator.evaluate(predictions)
    acc = acc_evaluator.evaluate(predictions)

    # Guarda o modelo e métricas
    models[label] = model
    evaluations[label] = {"AUC": auc, "Accuracy": acc}

    # Renomeia de volta a coluna label para o nome original
    test_df = test_df.withColumnRenamed("label", label)

# Exibe os resultados de avaliação
for label, metrics in evaluations.items():
    print(f"Resultados para {label}: AUC = {metrics['AUC']:.4f}, Accuracy = {metrics['Accuracy']:.4f}")

spark.stop()

Resultados para TWF: AUC = 0.9642, Accuracy = 0.9958
Resultados para HDF: AUC = 0.9978, Accuracy = 0.9917
Resultados para PWF: AUC = 0.9816, Accuracy = 0.9964
Resultados para OSF: AUC = 0.9997, Accuracy = 0.9969
Resultados para RNF: AUC = 0.4041, Accuracy = 0.9984
